In [1]:
import time
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

from pprint import pprint

from joblib import Parallel, delayed
import joblib

In [2]:
starttime = time.time()

In [3]:
refined_df = pd.read_csv('refined_df_mt500000_ur100.csv')

In [4]:
readTime = round((time.time() - starttime), 2)
readTime

0.09

In [5]:
knn_model = joblib.load('IMDbModel.h5')

In [6]:
refined_df = refined_df.drop('Unnamed: 0', axis=1)

In [7]:
movieUser_df = refined_df.pivot_table(
        index='userID',
            columns='titleID',
                ## Replacing all movies users haven't rated with a rating of 0
                values='rating').fillna(0)

In [8]:
movie_list = movieUser_df.columns
movieUser_scipy_df = csr_matrix(movieUser_df.values)

In [9]:
knn_model.fit(movieUser_scipy_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [10]:
simUsers = []
userDistances = []
highestMovies = []
recommendedMovies = []

In [11]:
def similar_users(user, n = 5):
    knn_input = np.asarray([movieUser_df.values[user-1]])
    distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
        
    for i in range(1,len(distances[0])):
        simUsers.append(indices[0][i]+1)
        userDistances.append(distances[0][i])
    return indices.flatten()[1:] + 1, distances.flatten()[1:]

In [12]:
def recommend_movies(n):
    n = min(len(mean_ratings_list),n)
    recommendedMovies.append(list(movie_list[np.argsort(mean_ratings_list)[::-1][:n]]))
    return recommendedMovies

In [13]:
target_user = 155
target_user_id = movieUser_df.index.values[target_user -1]
no_of_highest = 15
no_of_similar_users = 5
no_of_movies = 5

In [14]:
highestMovies.append(list(refined_df[refined_df['userID'] == target_user_id].sort_values('rating', ascending=False)['titleID'])[:no_of_highest])

In [15]:
similar_user_list, distance_list = similar_users(target_user, no_of_similar_users)
weighted_list = distance_list/np.sum(distance_list)
similar_user_ratings = movieUser_df.values[similar_user_list]
weighted_list = weighted_list[:,np.newaxis] + np.zeros(len(movie_list))
ratings_matrix = weighted_list*similar_user_ratings
mean_ratings_list = ratings_matrix.sum(axis =0)

In [16]:
dict1 = {'Users_Top_Movies': highestMovies, 'Similar_Users': simUsers, 'Sim_User_distances': userDistances, 'Recommendatons': recommendedMovies}
dict1

{'Users_Top_Movies': [['tt2084970',
   'tt0169547',
   'tt0372784',
   'tt2024544',
   'tt1049413',
   'tt0338013',
   'tt0421715',
   'tt0440963',
   'tt1504320',
   'tt0477348',
   'tt2582802',
   'tt0451279',
   'tt0167404',
   'tt0137523',
   'tt0120689']],
 'Similar_Users': [319, 1669, 1271, 2068, 1931],
 'Sim_User_distances': [0.49286152525930427,
  0.4933719746677129,
  0.49641122356793277,
  0.5067774389534012,
  0.5112792078308082],
 'Recommendatons': []}

In [17]:
totalTime = round((time.time() - starttime), 4)
totalTime

0.3966